In [ ]:
import subprocess
import imageio_ffmpeg
import os

In [4]:
def download_subway_video():
    url = "https://www.youtube.com/watch?v=RbVMiu4ubT0"
    
    command = (
        'yt-dlp '
        '-f "bv*+ba/b" '                          # Best video+audio combined (fallback: best)
        '--merge-output-format mp4 '              # Merge into MP4 if separate streams
        '--user-agent "Mozilla/5.0" '             # Bypass some 403 errors
        '--no-playlist '                          # Avoid playlist downloading
        '-o "subway.mp4" '                        # Output filename
        f'"{url}"'
    )
    
    os.system(command)


In [5]:
download_subway_video()

[youtube] Extracting URL: https://www.youtube.com/watch?v=RbVMiu4ubT0
[youtube] RbVMiu4ubT0: Downloading webpage
[youtube] RbVMiu4ubT0: Downloading tv client config
[youtube] RbVMiu4ubT0: Downloading tv player API JSON
[youtube] RbVMiu4ubT0: Downloading ios player API JSON
[youtube] RbVMiu4ubT0: Downloading m3u8 information
[info] RbVMiu4ubT0: Downloading 1 format(s): 303+251
[download] Destination: subway.f303.webm
[download] 100% of   73.48MiB in 00:00:06 at 11.64MiB/s    
[download] Destination: subway.f251.webm
[download] 100% of   64.54KiB in 00:00:00 at 1.12MiB/s     
[Merger] Merging formats into "subway.mp4"
Deleting original file subway.f303.webm (pass -k to keep)
Deleting original file subway.f251.webm (pass -k to keep)


In [6]:
def burn_audio_and_subtitles(video_path="subway.mp4", audio_path="output.wav", subtitle_path="output.srt", output_path="output_final.mp4"):
    # Check if input files exist
    if not os.path.exists(video_path):
        print(f"Video file not found: {video_path}")
        return
    if not os.path.exists(audio_path):
        print(f"Audio file not found: {audio_path}")
        return
    if not os.path.exists(subtitle_path):
        print(f"Subtitle file not found: {subtitle_path}")
        return
    
    ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()
    
    subtitle_style = (
        "FontName=Arial,"
        "FontSize=20,"
        "PrimaryColour=&Hffffff&,"
        "SecondaryColour=&Hffff00&,"
        "OutlineColour=&H000000&,"
        "BackColour=&H00000000&,"
        "BorderStyle=1,"
        "Outline=2,"
        "Shadow=1,"
        "MarginV=30,"
        "Alignment=2"
    )
    
    # Improved command with better audio handling and enhanced subtitle styling
    command = [
        ffmpeg_path,
        "-i", video_path,              # input video
        "-i", audio_path,              # input audio
        "-vf", f"subtitles={subtitle_path}:force_style='{subtitle_style}'",  # burn subtitles with enhanced styling
        "-c:v", "libx264",             # video codec
        "-c:a", "aac",                 # audio codec
        "-b:a", "192k",                # audio bitrate
        "-map", "0:v:0",               # map video from first input
        "-map", "1:a:0",               # map audio from second input (our generated audio)
        "-shortest",                   # stop when shortest stream ends
        "-y",                          # overwrite output file
        output_path
    ]


    
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print(f"Final video saved to {output_path}")
        
        # Check if output file was created and has reasonable size
        if os.path.exists(output_path):
            file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
            print(f"Output file size: {file_size:.1f} MB")
        else:
            print("Output file was not created")
            
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg error: {e}")
        print(f"Error output: {e.stderr}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Run it
burn_audio_and_subtitles()

🎬 Processing video with audio and subtitles...
📹 Video: subway.mp4
🔊 Audio: output.wav
📝 Subtitles: output.srt
🎯 Output: output_final.mp4
✨ Enhanced subtitle styling: Arial font, transparent background, word highlighting support
✅ Final video saved to output_final.mp4
📊 Output file size: 64.4 MB
